# 1. DATA PREPROCESSING STEP

In [35]:
#STEP 1 INSTALLING LIBRARIES
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import  LabelEncoder, OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

#step 2 load data
data = pd.read_csv('Churn_Modelling.csv')

#STEP 3: SEPARATE FEATURES (X) AND TARGET (y)
#we exclude rownumber,customerid,surname(indices 0,1,2)
#we take data from index 3 upto the last one as features
X=data.iloc[:,3:-1].values
#we take the last column as target
y=data.iloc[:,-1].values

# STEP 4: ENCODING CATEGORICAL DATA
#label encoding the gender column
#gender is at index 2 in our new x matrix (creditscore =0,geography=1,gender =2)
le=LabelEncoder()
X[:,2]= le.fit_transform(X[:,2])
#now female/male are 0/1
#one Hot Encoding the "geography" column
#"geography"is at index 1.it has three categories :french,spain ,germany
#we transform column1 into 3 separate binary columnc
ct = ColumnTransformer(transformers=[('encoder',OneHotEncoder(),[1])],remainder='passthrough')
X=np.array(ct.fit_transform(X))


#STEP 5 :SPLITINTO TRAIN AND TEST SET
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=0)

#step 6 :FEATURE SCALLING
sc=StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


#2 ARCHITECTURE OF THE BRAIN


In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#define the ANN ARCHITECTURE
class ChurnPredictor(nn.Module):
  def __init__(self):
    super(ChurnPredictor ,self).__init__()

    #hidden layer 1
    #input :12 features (from our preprocessed data)
    #output:8 neurons (arbitrary)
    self.layer1=nn.Linear(in_features =12,out_features=8)
    #hidden layer 2
    #input =8,output=8
    self.layer2=nn.Linear(in_features =8,out_features=8)
    #output latyer
    #input =8,output =1 as only probability is needed
    self.output_layer=nn.Linear(in_features=8,out_features=1)


  def forward(self,x):
    # 1. pass data through layer 1 and relu function
    x=F.relu(self.layer1(x))

    #2. Pass data through layer 2 and relu function
    x=F.relu(self.layer2(x))

    #3. pass through output layer and apply sigmoid  activation
    # sigmoid squashes the result between 0 and 1 (Probability)
    x=torch.sigmoid(self.output_layer(x))

    return x

# instantiate the model
model = ChurnPredictor()
print(model)


ChurnPredictor(
  (layer1): Linear(in_features=12, out_features=8, bias=True)
  (layer2): Linear(in_features=8, out_features=8, bias=True)
  (output_layer): Linear(in_features=8, out_features=1, bias=True)
)


# 3.TRAINING THE  BRAIN

In [37]:
#1. SETUP THE DATA OR PYTORCH
# convert standard numpy arrays into pytorch tensors
#.unsqueeze(1) changes y from [0,1,0] to [[0],[1],[0]]
X_train_tensor = torch.tensor(X_train,dtype= torch.float32)
y_train_tensor= torch.tensor(y_train,dtype=torch.float32).unsqueeze(1)

# 2. DEFINE THE TEACHER AND CORRECTOR
criterion=nn.BCELoss() #binary cross entropy loss (teacher)
optimizer=torch.optim.Adam(model.parameters(),lr=0.01) #adam optimiser (the corrector)

#3. THE TRAINING LOOP
epochs=100 # how many times we go through the dataset
for epochs in range(epochs):
  #A. FORWARD PASS (THE GUESS)
  y_pred=model(X_train_tensor)

  #B. CALCULATE LOSS(THE GRADE)
  loss=criterion(y_pred,y_train_tensor)
  #C. BACKWARD PASS(THE Learning )
  optimizer.zero_grad() #clear previous calculation
  loss.backward() #calculate gradients (how much to adjust each weight)
  optimizer.step() #update weights

  #monitoring
  if(epochs +1)%10 == 0 :
    print(f'Epoch [{epochs+1}/{epochs}] ,Loss:{loss.item():.4f}')











Epoch [10/9] ,Loss:0.6534
Epoch [20/19] ,Loss:0.4910
Epoch [30/29] ,Loss:0.4632
Epoch [40/39] ,Loss:0.4384
Epoch [50/49] ,Loss:0.4272
Epoch [60/59] ,Loss:0.4206
Epoch [70/69] ,Loss:0.4137
Epoch [80/79] ,Loss:0.4060
Epoch [90/89] ,Loss:0.3947
Epoch [100/99] ,Loss:0.3789


# 4 .THE EVALUATION PHASE

In [38]:
# PREPARE THE TEST DATA
X_test_tensor=torch.tensor(X_test, dtype=torch.float32)
y_test_tensor=torch.tensor(y_test,dtype=torch.float32).unsqueeze(1)

#2 evaluation mode
model.eval()

#the exam
with torch.no_grad():
  #make predictions
  y_pred_prob=model(X_test_tensor)

  #convert probabilty to yes or no
  #if prob >0.5,it rounds to 1 and if prob<0.5 it rounds to 0
  y_pred_cls=y_pred_prob.round()

  #grade the exam
  #.eq() compares predictions vs truth sum() counts the matches
  correct_count=y_pred_cls.eq(y_test_tensor).sum().item()
  accuracy = correct_count/y_test.shape[0]
  print(f"test accuracy :{accuracy:.4f}")


test accuracy :0.8225
